# MLOps Zoomcamp Homework 1

The goal of this homework is to train a simple model for predicting the duration of a ride.

- Module 1 Introduction  link: https://github.com/DataTalksClub/mlops-zoomcamp/tree/main/01-intro

In [61]:
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

In [45]:
df_01 = pd.read_parquet("./data/yellow_tripdata_2023-01.parquet")

In [46]:
df_01.shape

(3066766, 19)

In [47]:
len(df_01.columns)

19

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59

In [48]:
df_01['duration'] = df_01['tpep_dropoff_datetime'] - df_01['tpep_pickup_datetime']
df_01['duration'] = df_01['duration'].apply(lambda td: td.total_seconds() / 60)

In [49]:
df_01['duration'].std()

42.59435124195458

## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%

In [50]:
df_01_filtered = df_01[(df_01.duration >= 1) & (df_01.duration <= 60)]
df_01_filtered.shape

(3009173, 20)

In [51]:
filtered = df_01_filtered.shape[0]
total = df_01.shape[0]

print((filtered / total) * 100)

98.1220282212598


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

In [52]:
categorical = ['PULocationID', 'DOLocationID']

In [53]:
df_01_filtered['PULocationID'].value_counts(dropna=False)

PULocationID
237    147082
132    144529
236    137402
161    134047
186    108346
        ...  
30          2
58          2
109         1
221         1
245         1
Name: count, Length: 255, dtype: int64

In [54]:
df_01_filtered['DOLocationID'].value_counts(dropna=False)

DOLocationID
236    145424
237    131472
161    114979
170     87998
230     87755
        ...  
109        16
187        14
84         10
204         8
99          1
Name: count, Length: 260, dtype: int64

In [55]:
df_01_filtered[categorical] = df_01_filtered[categorical].astype(str)

/tmp/ipykernel_42019/3069845389.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_01_filtered[categorical] = df_01_filtered[categorical].astype(str)


In [56]:
train_dicts = df_01_filtered[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [57]:
X_train

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

In [58]:
len(dv.feature_names_)

515

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64

In [59]:
target = 'duration'
y_train = df_01_filtered[target].values

In [60]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

/home/rodrigoperes/miniconda3/envs/mlops-zoomcamp/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.649261931416412

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81